# Building Bentos

Bento 🍱 is a file archive with all the source code, models, data files and dependency configurations required for running a user-defined bentoml.Service, packaged into a standardized format.

While `bentoml.Service` standardizes the inference API definition, including the serving logic, runners initialization and API input, output types. Bento standardizes how to reproduce the required environment for running a `bentoml.Service` in production.

## The Build Command
A Bento can be created with the bentoml build CLI command with a `bentofile.yaml` build file. 

```yaml
service: "service:svc"  # Sames as the argument passed to 'bentoml serve'
labels:
    owner: bentoml-team
    stage: dev
include:
- "*.py"  # A pattern for matching which files to include in the bento
python:
    packages:  # Additional pip packages required by the service
    - scikit-learn
    - pandas
```

``` bash
$ bentoml build
Building BentoML service "iris_classifier:dpijemevl6nlhlg6" from build context "/home/user/gallery/quickstart"
Packing model "iris_clf:zy3dfgxzqkjrlgxi"
Locking PyPI package versions..
██████╗░███████╗███╗░░██╗████████╗░█████╗░███╗░░░███╗██╗░░░░░
██╔══██╗██╔════╝████╗░██║╚══██╔══╝██╔══██╗████╗░████║██║░░░░░
██████╦╝█████╗░░██╔██╗██║░░░██║░░░██║░░██║██╔████╔██║██║░░░░░
██╔══██╗██╔══╝░░██║╚████║░░░██║░░░██║░░██║██║╚██╔╝██║██║░░░░░
██████╦╝███████╗██║░╚███║░░░██║░░░╚█████╔╝██║░╚═╝░██║███████╗
╚═════╝░╚══════╝╚═╝░░╚══╝░░░╚═╝░░░░╚════╝░╚═╝░░░░░╚═╝╚══════╝
Successfully built Bento(tag="iris_classifier:dpijemevl6nlhlg6")
```
Similar to saving a model, a unique version tag will be automatically generated for the newly created Bento.


It is also possible to customize the Bento version string by specifying it in the `--version` CLI argument. However this is generally not recommended. Only use it if your team has a very specific naming convention for deployable artifacts,

```bash
$ bentoml build --version 1.0.1
```

## Managing Bentos

Bentos are the unit of deployment in BentoML, one of the most important artifact to keep track of for your model deployment workflow.

### Locaal Bento Store

*List*
```bash
$ bentoml list

Tag                               Size        Creation Time        Path
iris_classifier:nvjtj7wwfgsafuqj  16.99 KiB   2022-05-17 21:36:36  ~/bentoml/bentos/iris_classifier/nvjtj7wwfgsafuqj
iris_classifier:jxcnbhfv6w6kvuqj  19.68 KiB   2022-04-06 22:02:52  ~/bentoml/bentos/iris_classifier/jxcnbhfv6w6kvuqj
```

*Get*

```bash
» bentoml get iris_classifier:latest

service: service:svc
name: iris_classifier
version: nvjtj7wwfgsafuqj
bentoml_version: 1.0.0
creation_time: '2022-05-17T21:36:36.436878+00:00'
labels:
  owner: bentoml-team
  project: gallery
models:
- tag: iris_clf:nb5vrfgwfgtjruqj
  module: bentoml.sklearn
  creation_time: '2022-05-17T21:36:27.656424+00:00'
runners:
- name: iris_clf
  runnable_type: SklearnRunnable
  models:
  - iris_clf:nb5vrfgwfgtjruqj
  resource_config:
    cpu: 4.0
    nvidia_gpu: 0.0
apis:
- name: classify
  input_type: NumpyNdarray
  output_type: NumpyNdarray
```

*Delete*

```bash
» bentoml delete iris_classifier:latest -y

Bento(tag="iris_classifier:nvjtj7wwfgsafuqj") deleted
```

### Import and Export

Bentos can be exported to a standalone archive file outside of the store, for sharing Bentos betweem teams for moving between different deployment stages

```bash
$ bentoml export iris_classifier:latest .

INFO [cli] Bento(tag="iris_classifier:nvjtj7wwfgsafuqj") exported to ./iris_classifier-nvjtj7wwfgsafuqj.bento
```

```bash
$ bentoml import ./iris_classifier-nvjtj7wwfgsafuqj.bento

INFO [cli] Bento(tag="iris_classifier:nvjtj7wwfgsafuqj") imported
```

### Push and Pull

Yatai provides a centralized Bento repository that comes with flexible APIs and Web UI for managing all Bentos created by your team. It can be configured to store Bento files on cloud blob storage such as AWS S3, MinIO or GCS, and automatically build docker images when a new Bento was pushed.

``` bash
$ bentoml push iris_classifier:latest

Successfully pushed Bento "iris_classifier:nvjtj7wwfgsafuqj"
```

```bash
$ bentoml pull iris_classifier:nvjtj7wwfgsafuqj

Successfully pulled Bento "iris_classifier:nvjtj7wwfgsafuqj"
```

### Bento Management API

Python APIs are also provided for managing Bentos:

get
```python
import bentoml

bento = bentoml.get('iris_classifier:latest')

print(bento.tag)
print(bento.path)
print(bento.info.to_dict())
```

List
```python
import bentoml

bentos = bentoml.list()
```

Import / Export
```python
import bentoml

bentoml.export_bento('my_bento:latest', '/path/to/folder/my_bento.bento')
bentoml.import_bento('/path/to/folder/my_bento.bento')
```

Push / Pull

```python
import bentoml

bentoml.push("iris_classifier:nvjtj7wwfgsafuqj")
bentoml.pull("iris_classifier:nvjtj7wwfgsafuqj")
```

Delete

```python
import bentoml

bentoml.delete("iris_classifier:nvjtj7wwfgsafuqj")
```

### What's inside a Bento

It is possible to view the generated files in a specific Bento. Simply use the -o/--output option of the bentoml get command to find the file path to the Bento archive directory.

```shell
» cd $(bentoml get iris_classifier:latest -o path)
» tree
.
├── README.md
├── apis
│   └── openapi.yaml
├── bento.yaml
├── env
│   ├── docker
│   │   ├── Dockerfile
│   │   └── entrypoint.sh
│   └── python
│       ├── requirements.lock.txt
│       ├── requirements.txt
│       └── version.txt
├── models
│    └── iris_clf
│       ├── latest
│       └── nb5vrfgwfgtjruqj
│           ├── model.yaml
│           └── saved_model.pkl
└── src
    ├── locustfile.py
    ├── service.py
    └── train.py
```

- `src` directory contains files specified under the include field in the bentofile.yaml. These files are relative to user Python code’s CWD (current working directory), which makes importing relative modules and file path inside user code possible.

- `models` directory contains all models required by the Service. This is automatically determined from the bentoml.Service object’s runners list.

- `apis` directory contains all API definitions. This directory contains API specs that are generated from the bentoml.Service object’s API definitions.

- `env` directory contains all environment-related files which will help boostrap the Bento 🍱. This directory contains files that are generated from Bento Build Options that is specified under bentofile.yaml.

## Bento Build Options

Build options are specified in a `.yaml` file, which customizes the final Bento produced.

By convention, this file is named `bentofile.yaml`.

### Service
`service` is a **required** field which specifies where the `bentoml.Service` object is defined.
- `service` refers to the Python module (the `service.py` file)
- `svc` refers to the `bentoml.Service` object created in `service.py`, with `svc=bentoml.Service(...)`

### Description

`description` field allows to customize documentation for any given Bento.

The diescription contents must be plain text, optionally in Markdown format. Description can be specified either inline in the `bentofil.yaml`, or via a file path to an existing text file:

**Inline**
```yaml
service: "service.py:svc"
description: |
    ## Description For My Bento
    
    Use **any markdown syntax** here!
    
    > BentoML is awesome!
include:
...
```
**File path**
```yaml
service: "service.py:svc"
description: "file: ./README.md"
include:
...
```

### Labels
`labels` are key-value pairs that are attached to an object.
In BentoML, both `bento` and `Model` can have labels attached to them. Labels are intended to be used to specify identifed attributes of Bentos/Models that are meaningful and relevant to users, but do not directly imply semantics to the rest of the system.

```yaml
labels:
    owner: bentoml-team
    stage: not-ready
```

### Files to include & exclude
include or exclude wildcard and directory pattern matching
```yaml
...
include:
    - "data/"
    - "**/*.py"
    - "config/*.json"
    - "path/to/a/file.csv"
exclude:
    - "test/"
    - "secrets.key"
```

Users can also opt to place a .bentoignore file in the build_ctx directory. This is what a .bentoignore file would look like:
```yaml
.bentoignore

__pycache__/
*.py[cod]
*$py.class
.ipynb_checkpoints/
training_data/
```

### Python Packages
Required Python packages for a given Bento can be specified under the `python.packages` field.

When a package name is left without a version, BentoML will lock the package to the version available under the current environment when running `bentoml build`. User can also specify the desired version, install from a custom PyPI source, or install from a GitHub repo

```yaml
python:
    packages:
    - "numpy"
    - "matplotlib==3.5.1"
    - "package>=0.2,<0.3"
    - "torchvision==0.9.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cpu"
    - "git+https://github.com/username/mylib.git@main"
```

To use a variant of BentoML with additional features such as gRPC, tracing exporters, pydantic validation, specify the desired variant in the under python.packages field:

```yaml
python:
    packages:
    - "bentoml[grpc]"   # gRPC
    - "bentoml[aws]"    # AWS
    - "bentoml[io-json]"   # JSON IO
    - "bentoml[io-image]"  # IMAGE IO
    - "bentoml[io-pandas]" # Pandas IO
    - "bentoml[io-json]"   # JSON IO
    - "bentoml[tracing-jaeger]"  # Jaeger
    - "bentoml[tracing-zipkin]"  # Zipkin
```

If you already have a requirements.txt file that defines python packages for your project, you may also supply a path to the requirements.txt file directly:
```yaml
python:
    requirements_txt: "./project-a/ml-requirements.txt"
```

#### Pip Install Options

```yaml
python:
    requirements_txt: "./requirements.txt"
    index_url: "https://my.mirror.com/simple"
    no_index: False
    trusted_host:
    - "pypi.python.org"
    - "my.mirror.com"
    find_links:
    - "https://download.pytorch.org/whl/cu80/stable.html"
    extra_index_url:
    - "https://<other api token>:@my.mirror.com/pypi/simple"
    - "https://pypi.python.org/simple"
    pip_args: "--pre -U --force-reinstall"
```

#### PyPI Package Locking

By default, BentoML automatically locks all package versions, as well as all packages in their dependency graph, to the version found in the current build environment, and generates a `requirements.lock.txt` file

```yaml
python:
    requirements_txt: "requirements.txt"
    lock_packages: false
```

#### Python WHeels

Python .whl files are also supported as a type of dependency to include in a Bento.
```yaml
python:
    wheels:
    - ./lib/my_package.whl
```